In [0]:
import numpy as np
import tensorflow as tf

# スケジューリング

In [0]:
initial_learning_rate = 0.1
decay_steps = 10000
decay_rate = 1/10
glocal_step = tf.Variable(0, trainable=False, name = "global_step")
learning_rate = tf.train.exponential_decay(initial_learning_rate, global_step, decay_steps, decay_rate)
optimizer = tf.train.MomentumOptimizer(learning_rate, momentum=0.9)
training_op = optimizer.minimize(loss, global_step = global_step)

# l1,l2正則化

In [0]:
scale = 0.001

my_dense_layer = partial(tf.layers.dense, activation = tf.nn.relu, kernel_regularizer=tf.contrib.layers.l1_regularizer(scale))

with tf.name_scope("dnn"):
  hidden1 = my_dense_layer(X, n_hidden1, name = "hidden1")
  hidden2 = my_dense_layer(hidden1, n_hidden2, name = "hidden2")
  logits = my_dense_layer(hidden2, n_outputs, activation = None, name = "outputs")

with name_scope("loss"):
  xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, logits = logits)
  
  base_loss = tf.reduce_mean(xentropy, name = "avg_xentropy")
  reg_losses = tf.reduce_sum(tf.abs())


reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
loss = tf.add_n([base_loss] + reg_losses, name = "loss")

In [0]:
training = tf.placeholder_with_default(False, shape = (), name = "training")
dropout_rate = 0.5

In [0]:
X_drop = tf.layers.dropout(X, dropout_rate, training = training)

NameError: ignored

In [0]:
with tf.name_scope("dnn"):
  hidden1 = tf.layers.dense(X_drop, n_hidden1, activation = tf.nn.relu)
  hidden1_drop = tf.layers.dropout(hidden1, dropout_rate, training = training)
  hidden2 = tf.layers.dense(hidden1_drop, n_hidden2, activation = tf.nn.relu)
  hidden2_drop = tf.layers.dropout(hidden2, dropout_rate, training = training)
  logits = tf.layers.dense(hidden2_drop, n_outputs, name = "logits")

In [0]:
def max_norm_regularizer(threshold, axes = 1, name = "max_norm", collection = "max_norm"):
  
  def max_norm(weights):
    clipped = tf.clip_by_norm(weights, clip_norm=threshold, axes = axes)
    clip_weights = tf.assign(weights, clipped, name = name)
    tf.add_to_collection(collection, clip_weights)
    return None
  
  return max_norm

In [0]:
max_norm_reg = max_norm_regularizer(theshold=1.0)

with tf.name_scope("dnn"):
  hidden1 = tf.layers.dense(X, n_hidden1, activation = tf.nn.relu, kernel_regularizer=max_norm_reg, name = "hidden1")
  hidden2 = tf.layers.dense(hidden1, n_hidden2, activation = tf.nn.relu, kernel_regularizer = max_norm_reg, name = "hidden2")
  logits = tf.layers.dense(hidden2, n_outputs, name = "outputs")

In [0]:
clip_all_weights = tf.get_collection("max_norm")

with tf.Session() as sess:
  init.run()
  for epoch in range(n_epochs):
    for iteration in range(mnis.train.num):
      sess.run(training_op, feed_dict = {X:X_batch, y:y_batch})
      sess.run(clip_all_weights)